In [ ]:
!apt-get update -qq
!apt-get install -y -qq libopenmpi-dev openmpi-bin
!pip install mpi4py

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [ ]:
%%writefile montecarlo.hpp
#pragma once
#include <random>

inline long long monte_carlo_pi(long long num_points, unsigned int seed) {
    std::mt19937_64 gen(seed);  // 64-bit Mersenne Twister
    std::uniform_real_distribution<double> dist(0.0, 1.0);
    long long count = 0;

    for (long long i = 0; i < num_points; ++i) {
        double x = dist(gen);
        double y = dist(gen);
        if (x * x + y * y <= 1.0) {
            ++count;
        }
    }

    return count;
}


Overwriting montecarlo.hpp


In [ ]:
%%writefile montecarlo_seq.cpp
#include <iostream>
#include <chrono>
#include "montecarlo.hpp"

int main() {
    long long num_points = 1e9;  // Increase to make it more expensive
    unsigned int seed = 42;

    auto start = std::chrono::high_resolution_clock::now();
    int inside_circle = monte_carlo_pi(num_points, seed);
    auto end = std::chrono::high_resolution_clock::now();

    double pi_estimate = 4.0 * inside_circle / num_points;
    std::chrono::duration<double> duration = end - start;

    std::cout << "Estimated Pi (Sequential): " << pi_estimate << "\n";
    std::cout << "Time taken: " << duration.count() << " seconds\n";

    return 0;
}


Overwriting montecarlo_seq.cpp


In [ ]:
!g++ montecarlo_seq.cpp -o montecarlo_seq
!./montecarlo_seq

Estimated Pi (Sequential): 3.14156
Time taken: 115.004 seconds


In [ ]:
%%writefile montecarlo_mpi.cpp
#include <mpi.h>
#include <iostream>
#include "montecarlo.hpp"

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    const long long total_points = 1e9;
    const long long local_points = total_points / size;

    double start_time = MPI_Wtime();

    long long local_count = monte_carlo_pi(local_points, 42 + rank);

    long long global_count = 0;
    MPI_Reduce(&local_count, &global_count, 1, MPI_LONG_LONG, MPI_SUM, 0, MPI_COMM_WORLD);

    double end_time = MPI_Wtime();

    if (rank == 0) {
        double pi_estimate = 4.0 * global_count / total_points;
        std::cout << "Estimated Pi (MPI): " << pi_estimate << "\n";
        std::cout << "Time taken: " << (end_time - start_time) << " seconds\n";
    }

    MPI_Finalize();
    return 0;
}


Overwriting montecarlo_mpi.cpp


In [ ]:
!mpic++ montecarlo_mpi.cpp -o montecarlo_mpi
!mpirun --allow-run-as-root --oversubscribe -np 4 ./montecarlo_mpi

Estimated Pi (MPI): 3.14164
Time taken: 84.6947 seconds


In [ ]:
%%writefile montecarlo_omp.cpp
#include <iostream>
#include <random>
#include <omp.h>
#include <chrono>

int main() {
    const long long num_points = 1e9;
    long long count = 0;

    auto start = std::chrono::high_resolution_clock::now();

    #pragma omp parallel reduction(+:count)
    {
        std::mt19937 gen(42 + omp_get_thread_num()); // Thread-local RNG
        std::uniform_real_distribution<double> dist(0.0, 1.0);

        long long local_count = 0;

        #pragma omp for schedule(static)
        for (long long i = 0; i < num_points; ++i) {
            double x = dist(gen);
            double y = dist(gen);
            if (x * x + y * y <= 1.0) {
                ++local_count;
            }
        }

        count += local_count;  // implicit in reduction
    }

    auto end = std::chrono::high_resolution_clock::now();
    double pi_estimate = 4.0 * count / num_points;
    std::chrono::duration<double> duration = end - start;

    std::cout << "Estimated Pi (OpenMP): " << pi_estimate << "\n";
    std::cout << "Time taken: " << duration.count() << " seconds\n";

    return 0;
}


Overwriting montecarlo_omp.cpp


In [ ]:
!g++ -fopenmp -O3 montecarlo_omp.cpp -o montecarlo_omp
!./montecarlo_omp


Estimated Pi (OpenMP): 3.14156
Time taken: 29.0603 seconds


In [ ]:
%%writefile montecarlo_cuda.cu
#include <iostream>
#include <curand_kernel.h>
#include <cuda.h>

__global__ void montecarlo_kernel(unsigned int* counts, int points_per_thread, unsigned long seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    curandState state;
    curand_init(seed, idx, 0, &state);

    unsigned int local_count = 0;
    for (int i = 0; i < points_per_thread; ++i) {
        float x = curand_uniform(&state);
        float y = curand_uniform(&state);
        if (x * x + y * y <= 1.0f) {
            ++local_count;
        }
    }
    counts[idx] = local_count;
}

void checkCuda(cudaError_t result, const char* msg) {
    if (result != cudaSuccess) {
        std::cerr << "CUDA Error (" << msg << "): " << cudaGetErrorString(result) << std::endl;
        exit(EXIT_FAILURE);
    }
}

int main() {
    const long long total_points = 1e8;
    const int threads_per_block = 256;
    const int num_blocks = 256;
    const int num_threads = threads_per_block * num_blocks;
    const int points_per_thread = total_points / num_threads;

    unsigned int* d_counts;
    unsigned int* h_counts = new unsigned int[num_threads];

    checkCuda(cudaMalloc(&d_counts, num_threads * sizeof(unsigned int)), "cudaMalloc");

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    montecarlo_kernel<<<num_blocks, threads_per_block>>>(d_counts, points_per_thread, 42);
    checkCuda(cudaGetLastError(), "Kernel Launch");
    checkCuda(cudaDeviceSynchronize(), "Kernel Sync");

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    checkCuda(cudaMemcpy(h_counts, d_counts, num_threads * sizeof(unsigned int), cudaMemcpyDeviceToHost), "cudaMemcpy");

    unsigned long long total_in_circle = 0;
    for (int i = 0; i < num_threads; ++i) {
        total_in_circle += h_counts[i];
    }

    double pi_estimate = 4.0 * total_in_circle / total_points;
    std::cout << "Estimated Pi (CUDA): " << pi_estimate << "\n";
    std::cout << "Time taken: " << milliseconds / 1000.0 << " seconds\n";

    cudaFree(d_counts);
    delete[] h_counts;
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Overwriting montecarlo_cuda.cu


In [ ]:
!nvcc -arch=sm_75 -O3 montecarlo_cuda.cu -o montecarlo_cuda
!./montecarlo_cuda


Estimated Pi (CUDA): 3.13964
Time taken: 0.00341619 seconds


In [ ]:
import matplotlib.pyplot as plt

# Execution times in seconds
time_seq = 407.21
time_mpi = 305.4391
time_omp = 0.969471
time_cuda = 0.362703

labels = ['Sequential (Python)', 'OpenMP (C++)', 'CUDA (GPU)', 'MPI (Distributed)']
times = [time_seq, time_omp, time_cuda, time_mpi]
speedups = [time_seq / t for t in times]

# 🎨 Start plotting
plt.figure(figsize=(18, 5))

# 📊 Plot 1: Bar chart for Execution Time (Log Scale)
plt.subplot(1, 3, 1)
bars = plt.bar(labels, times, color=['gray', 'skyblue', 'orange', 'lightgreen'])
plt.yscale('log')
plt.ylabel('Execution Time (s) [Log Scale]')
plt.title('Execution Time (Lower is Better)')
for bar, val in zip(bars, times):
    plt.text(bar.get_x() + bar.get_width()/2, val * 1.2, f"{val:.2f}s", ha='center')

# 📈 Plot 2: Line plot for Speedup
plt.subplot(1, 3, 2)
plt.plot(labels, speedups, marker='o', linestyle='-', color='crimson')
plt.ylabel('Speedup (×)')
plt.title('Speedup vs Sequential')
for i, s in enumerate(speedups):
    plt.text(i, s + 0.5, f"{s:.2f}×", ha='center')

# 🥧 Plot 3: Pie chart for time proportion
plt.subplot(1, 3, 3)
plt.pie(times, labels=labels, autopct='%1.1f%%', colors=['gray', 'skyblue', 'orange', 'lightgreen'])
plt.title('Proportion of Total Runtime')

plt.tight_layout()
plt.show()
